In [1]:
import os
import re
import altair as alt
import pandas as pd

def getResults(shape, getInProgress=False, getCancelled=False):
    for root, _, fs in os.walk(shape):
        files =  [os.path.join(root, f) for f in fs]
        break
    results = []
    inProgressCounter = 0
    cancelledCounter = 0
    for path in files:
        with open(path) as f:
            result = f.readlines()[-7].strip()
        if result == '=========================================================':
            inProgressCounter += 1
            if not getInProgress:
                continue
            result = 'In progress'
        if result.startswith('Job submitted date = '):
            cancelledCounter += 1
            if not getCancelled:
                continue
            result = "Cancelled"
        if result == '---------------':
            result = 'Error'
        category = result
        if not result in ['Cancelled', 'UND', 'None', 'Error', 'TIMEOUT', 'In progress']:
            category = 'Solution'

        m = re.search('([0-9]+)t_([0-9]+)c-([0-9]+).out', path)
        m.group(0)
        results.append({
            'nCubeTypes': int(m.group(1)),
            'nColors': int(m.group(2)),
            'jobID': int(m.group(3)),
            'result': result,
            'category': category
        })
    df = pd.DataFrame(results)
    df.to_csv(shape+'.csv')
    print("Found {} in progress and {} cancelled".format(inProgressCounter, cancelledCounter))
    return df

def plotData(df):
    return alt.Chart(df).transform_calculate(
        url='https://akodiat.github.io/polycubes?rule=' + alt.datum.result
    ).mark_square().encode(
        alt.X('nCubeTypes:Q', title='# of species'),
        alt.Y('nColors:Q', title='# of colors'),
        color='category',
        tooltip=['result', 'nCubeTypes', 'nColors', 'jobID'],
        href='url:N'
    )

In [2]:
plotData(getResults('cube')).properties(
    title='Cube'
)

Found 0 in progress and 304 cancelled


alt.Chart(...)

In [3]:
plotData(getResults('filled_cube', True, False)).properties(
    title='Filled cube'
)

Found 39 in progress and 717 cancelled


alt.Chart(...)

In [4]:
plotData(getResults('chair')).properties(
    title='Chair'
)

Found 370 in progress and 0 cancelled


alt.Chart(...)

In [50]:
def getInProgressOverCompl(name, maxNC, maxNT):
    return getResults(name, getInProgress=True).query(
        "category == 'In progress' and (nCubeTypes > {} or nColors > {})".format(
            maxNC, maxNT
    ))['jobID'].values.tolist()

In [63]:
print(' '.join(["{}".format(id) for id in getInProgressOverCompl('filled_cube', 11,11)]))

Found 39 in progress and 717 cancelled



In [25]:
def getInProgress(name):
    return getResults(name, getInProgress=True).query("category == 'In progress'")['jobID'].values.tolist()

In [31]:
print(' '.join(["{}".format(id) for id in getInProgress('filled_cube')]))

1563340 1563376 1563622 1563428 1563605 1563639 1563614 1563477 1563512 1563453 1563503 1563474 1563473 1563442 1563413 1563497 1563286 1563410 1563578 1563390 1563596 1563571 1563527 1563254 1563395 1563285 1563377 1563628 1563518 1563626 1563572 1563650 1563648 1563416 1563239 1563495 1563299 1563603 1563536 1563612 1563435 1563457 1563552 1563642 1563471 1563528 1563576 1563604 1563389 1563629 1563615 1563341 1563561 1563443 1563553 1563631 1563502 1563560 1563643 1563372 1563554 1563496 1563411 1563478 1563597 1563429 1563412 1563624 1563644 1563479 1563325 1563635 1563476 1563455 1563298 1563638 1563311 1563623 1563284 1563392 1563584 1563581 1563323 1563649 1563630 1563357 1563579 1563480 1563494 1563458 1563651 1563652 1563451 1563577 1563546 1563526 1563602 1563613 1563545 1563472 1563636 1563452 1563449 1563519 1563355 1563627 1563391 1563463 1563381 1563432 1563263 1563506 1563409 1563264 1563573 1563558 1563431 1563464 1563312 1563511 1563456 1563547 1563396 1563599 1563523 